#### Importing tfds library

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [ ]:
tfds.load('eurosat')

In [ ]:
ds, ds_info = tfds.load('eurosat/rgb',
                        with_info=True,
                        split='train',
                        data_dir='/content/drive/My Drive/Colab Notebooks/Florence/sample_data/')

In [ ]:
print(ds)

In [ ]:
ds.output_shapes

In [ ]:
dataset = ds.from_tensors([1,2,3])

In [ ]:
DATA_DIR='/content/drive/My Drive/Colab Notebooks/Florence/sample_data1/'

In [ ]:
ds_train, ds_valid = tfds.load('eurosat/rgb',
                               split=['train[:70%]', 'train[70%:]'],
                               data_dir=DATA_DIR,
                               as_supervised=True)

#### Eurosat example images

In [ ]:
tfds.show_examples(ds_info,ds,rows=5,cols=5)

### **Constructing a PIPELINE**

In [ ]:
BATCH_SIZE = 16
AUTO = tf.data.experimental.AUTOTUNE
SHUFFLE_BUFFER = int(ds_info.splits['train'].num_examples * 0.7)

ds_train, ds_valid = tfds.load('eurosat/rgb',
                               split=['train[:70%]', 'train[70%:]'],
                               data_dir=DATA_DIR,
                               as_supervised=True)

def preprocess(image, label):
    image = tf.image.convert_image_dtype(image, dtype=tf.float32)
    return image, label

ds_train = (ds_train
            .map(preprocess, AUTO)
            .cache()
            .shuffle(SHUFFLE_BUFFER)
            .repeat()
            # Augmentations go here .map(augment, AUTO)
            .batch(BATCH_SIZE, drop_remainder=True)
            .prefetch(AUTO))

ds_valid = (ds_valid
            .map(preprocess, AUTO)
            .cache()
            .batch(BATCH_SIZE)
            .prefetch(AUTO))

### **Define Model**

In [ ]:
import tensorflow.keras as keras
import tensorflow.keras.layers as layers

NUM_CLASSES = ds_info.features['label'].num_classes

model = keras.Sequential([
    layers.BatchNormalization(),
    layers.Conv2D(filters=16, kernel_size=5, padding='same', activation='elu'),
    layers.MaxPool2D(),
    
    layers.BatchNormalization(),
    layers.Conv2D(32, 3, padding='same', activation='elu'),
    layers.Conv2D(32, 3, padding='same', activation='elu'),
     layers.MaxPool2D(),
    
    layers.BatchNormalization(),
    layers.Conv2D(64, 3, padding='same', activation='elu'),
    layers.Conv2D(64, 3, padding='same', activation='elu'),
    layers.MaxPool2D(),
    
    layers.Flatten(),
    layers.Dense(128, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(64, activation='elu'),
    layers.Dropout(0.5),
    layers.Dense(NUM_CLASSES, activation='softmax')
])
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['sparse_categorical_accuracy'],
)

#### Training the model

In [ ]:
EPOCHS = 50
STEPS_PER_EPOCH = int(ds_info.splits['train'].num_examples * 0.7) // BATCH_SIZE

early_stopping = tf.keras.callbacks.EarlyStopping(patience=7, min_delta=0.001, restore_best_weights=True)

history = model.fit(
    ds_train,
    validation_data=ds_valid,
    epochs=EPOCHS,
    steps_per_epoch=STEPS_PER_EPOCH,
    callbacks=[early_stopping],
)

In [ ]:
import pandas as pd

history_frame = pd.DataFrame(history.history)
history_frame.loc[:, ['loss', 'val_loss']].plot()
history_frame.loc[:, ['sparse_categorical_accuracy', 'val_sparse_categorical_accuracy']].plot();

### **Saving the model**

In [ ]:
model.save("/content/drive/Shared drives/Coastal_Image_Analysis/Data/eurosat_model.h5")

In [ ]:
import numpy as np


#### Importing images

In [ ]:
data = np.load('/content/drive/Shared drives/Coastal_Image_Analysis/Data/64resizedimagearray.npz')

In [ ]:
data.files

In [ ]:
data = data['arr_0']

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(data[2])

In [ ]:
data.shape

In [ ]:

import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import glob
import cv2
from sklearn.metrics import silhouette_score
from tensorflow.keras import models
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from google.colab.patches import cv2_imshow
import os
import random
import shutil

Loading the model and popping the last layers out

In [ ]:
new_model = tf.keras.models.load_model('/content/drive/Shared drives/Coastal_Image_Analysis/Data/eurosat_model.h5')


In [ ]:
 new_model.summary()

In [ ]:
new_model.pop()
new_model.pop()
new_model.pop()
new_model.pop()
new_model.pop()
new_model.pop()

#### Modified model

In [ ]:
new_model.summary()

### **Predicting our data based on our saved and loaded model**

In [ ]:
  predictions = new_model.predict(data.reshape(-1, 64, 64, 3))


In [ ]:
pred_images = predictions.reshape(data.shape[0], -1)

In [ ]:

import math
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import glob
import cv2
from sklearn.metrics import silhouette_score
from tensorflow.keras import models
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import tensorflow.keras.backend as K
from google.colab.patches import cv2_imshow
import os
import random
import shutil

Silhoutte Score for number optimal K

In [ ]:
sil = []
kl = []
kmax = 10
for k in range(2, kmax+1):
    kmeans2 = KMeans(n_clusters = k).fit(pred_images)
    labels = kmeans2.labels_
    sil.append(silhouette_score(pred_images, labels, metric =   'euclidean'))
    kl.append(k)

In [ ]:
plt.plot(kl, sil)
plt.ylabel('Silhoutte Score')
plt.ylabel('K')
plt.show()

Elbow method for optimal K

In [ ]:
Sum_of_squared_distances = []
K = range(1,20)
for k in K:
    km = KMeans(n_clusters=k)
    km = km.fit(pred_images)
    Sum_of_squared_distances.append(math.log(km.inertia_))

In [ ]:
#13-19
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.ylabel('log of Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

#### Performing K-means clustering

In [ ]:
k = 9
kmodel = KMeans(n_clusters = k, n_jobs=-1, random_state=728)
kmodel.fit(pred_images)
kpredictions = kmodel.predict(pred_images)
#shutil.rmtree('output')

In [ ]:
k_means_labels = kmodel.labels_ #List of labels of each dataset
print("The list of labels of the clusters are " + str(np.unique(k_means_labels)))

In [ ]:
G = len(np.unique(k_means_labels)) #Number of labels

#2D matrix  for an array of indexes of the given label
cluster_index= [[] for i in range(G)]
for i, label in enumerate(k_means_labels,0):
    for n in range(G):
        if label == n:
            cluster_index[n].append(i)
        else:
            continue

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline


#### Displaying Images from different clusters

In [ ]:
#Visualisation for clusters = clust
clust = 9
num = 9 #num of data to visualize from the cluster
for j in range(0,clust):
  plt.figure(figsize=(20,20))
  #plt.clf()
  print('Cluster '+str(j))
  for i in range(1,num+1):
    plt.subplot(7, 7, i) #(Number of rows, Number of column per row, item number)
    plt.imshow(data[cluster_index[j][i]].reshape(64,64,3))     
  plt.show()

#### Gradmap Function

In [ ]:
pip install cv2_plt_imshow

In [ ]:
from google.colab.patches import cv2_imshow
from cv2_plt_imshow import cv2_plt_imshow,plt_format
clust=8
n=3
def gradmap():
  for num,j in enumerate(['Zero','One','Two','Three','Four','Five','Six','Seven','Eight']):
    #p=path+j
    #files=os.listdir(p)
    print("Cluster "+j)
    for i in range(n):
      d=random.choice(range(len(cluster_index[num])))
      #new_img = image.load_img(p+"/"+d, target_size=(224, 224))
      new_img_tensor = data[cluster_index[num][d]]
      new_img_tensor_x1 = np.expand_dims(new_img_tensor, axis=0)
      new_img_tensor_x1.shape
      with tf.GradientTape() as tape:
        layers = [layer.name for layer in new_model.layers if isinstance(layer, tf.keras.layers.Conv2D)]
        last_conv_layer = new_model.get_layer(layers[-1])
        iterate = tf.keras.models.Model([new_model.inputs], [new_model.output, last_conv_layer.output])
        model_out, last_conv_layer = iterate(new_img_tensor_x1)
      #class_out = model_out[:, np.argmax(model_out[0])]
        grads = tape.gradient(model_out,last_conv_layer)
        pooled_grads = K.mean(grads, axis=(0, 1, 2))
      heatmap = tf.reduce_mean(tf.multiply(pooled_grads, last_conv_layer), axis=-1)
      heatmap = np.maximum(heatmap, 0)
      heatmap /= np.max(heatmap)
      heatmap = heatmap.reshape((16, 16))
      #plt.matshow(heatmap)
      #plt.show()
      new_img_org = data[cluster_index[num][d]]
      INTENSITY = 0.5
      heatmap = cv2.resize(heatmap, (new_img_org.shape[1], new_img_org.shape[0]))
      heatmap = cv2.applyColorMap(np.uint8(255*heatmap), cv2.COLORMAP_JET)
      new_img = heatmap * INTENSITY + new_img_org

      #im=cv2_imshow(new_img)
      
      #scale_percent = 40 # percent of original size
      #width = int(new_img.shape[1] * scale_percent / 100) 
      #height = int(new_img.shape[0] * scale_percent / 100) 
      #dim = (width, height) 

      # resize image
      #resized = cv2.resize(new_img, dim, interpolation = cv2.INTER_AREA) 
      #print(new_img.shape)
      #img_float32 = np.float32(new_img)
      #plt.figure(figsize=(50,50))
      
      plt.imshow(data[cluster_index[num][d]])
      plt.show()
      cv2_plt_imshow(new_img.astype(np.uint8))
      plt.show()
      #cv2.waitKey()
    
 
    

#### Displaying gradmap image with original image

In [ ]:
gradmap()

### TSNE

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
data.shape

In [ ]:
data2=np.array(np.float32(pred_images).reshape(len(pred_images), -1)/255)

In [ ]:
tsne = TSNE().fit_transform(data2)


In [ ]:
tsne3d=TSNE(n_components=3).fit_transform(data2)

In [ ]:
import pandas as pd



In [ ]:
tsne_components = pd.DataFrame(tsne,columns=['comp1','comp2'])

In [ ]:
tsne_components3d = pd.DataFrame(tsne3d,columns=['comp1','comp2','comp3'])

In [ ]:
tsne_components['cluster']=k_means_labels

In [ ]:
tsne_components3d['cluster']=k_means_labels

In [ ]:
tsne_components3d.head()

### TSNE visualisation using plotly

In [ ]:
import plotly.offline as py
from plotly.offline import iplot, init_notebook_mode
#py.init_notebook_mode(connected=True)
#py.init_notebook_mode()
import plotly.graph_objs as go
import plotly.tools as tls
import plotly.express as px

In [ ]:

tsne_components["cluster"] = tsne_components["cluster"].astype(str)
fig = px.scatter(tsne_components, x="comp1", y="comp2", color="cluster",
                  )

fig.show()

In [ ]:
tsne_components3d["cluster"] = tsne_components3d["cluster"].astype(str)
fig = px.scatter_3d(tsne_components3d, x="comp1", y="comp2",z="comp3", color="cluster",
                  )


fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
fig.show()

In [ ]:
#Visualisation for clusters = clust
clust = 9
num = 3 #num of data to visualize from the cluster
for j in range(0,clust):
  plt.figure(figsize=(30,30))
  #plt.clf()
  print('Cluster '+str(j))
  for i in range(1,num+1):
    plt.subplot(7, 7, i) #(Number of rows, Number of column per row, item number)
    plt.imshow(data[cluster_index[j][i]].reshape(64,64,3))     
  plt.show()